# Project Asclepius v2.0: An AI-Powered Semantic Layer for Proactive Healthcare

# AI-Driven Clinical Performance Insights

## What is This Project?

Our solution generates new clinical Key Performance Indicators (KPIs) that go beyond traditional rule-based scoring by analyzing unstructured clinical notes. These novel metrics include:

- **Narrative Risk Score (NRS):** A 0-100 risk score derived from the full patient narrative, capturing a holistic view of patient risk.
- **Care Fragmentation Index (CFI):** Measures the level of fragmentation or poor coordination in care across different providers.
- **Inferred Social Determinant Burden (ISDB):** Detects social barriers like housing instability or transportation issues directly from text.

## Why is it Important?

Traditional healthcare analytics rely heavily on structured codes and rules, which often overlook the crucial context embedded in free-text clinical notes. This creates **"data obscurity,"** where hidden risks remain invisible, leading to reactive, inefficient, and costly care. Our project directly tackles this problem by unlocking insights from overlooked data.

## How Does it Work?

This project is a showcase of the **AI Architect** and **Semantic Detective** approaches, built entirely within BigQuery.

- **Integration:** Harmonizes multimodal patient data (notes, labs, demographics) into a unified, chronological view.
- **AI-Powered NLP:** Uses BigQuery's `ML.GENERATE_TEXT` with Gemini to extract clinical entities, social factors, and coordination issues from clinical narratives.
- **Aggregated KPIs:** Transforms these AI-driven insights into patient-level KPIs that quantify risk, social challenges, and care quality.
- **Semantic Search:** Leverages `ML.GENERATE_EMBEDDING` and `VECTOR_SEARCH` for clinicians to find similar patients based on narrative context, improving diagnostic and treatment planning.
- **Operationalized Insights:** Delivers alerts and dashboards designed for proactive, patient-centric care delivery.

## What is the Result?

Starting with raw, fragmented, and noisy Electronic Health Record (EHR) data, the system generates high-quality, AI-powered KPIs and semantic search tools that uncover hidden risks. This empowers clinicians and care teams to intervene earlier, improve coordination, and address social barriers. We transform healthcare analytics from rigid, top-down systems into flexible, data-driven discovery engines.

### Core Innovation: AI-Derived KPIs
- **Narrative Risk Score (NRS)**: A 0-100 risk quantification from clinical narratives.
- **Care Fragmentation Index (CFI)**: Measures provider coordination breakdowns.
- **Inferred Social Determinant Burden (ISDB)**: Identifies non-clinical barriers from text.

---

## Prerequisites
Before running this notebook, ensure you have:
1. ✅ A GCP Project with the BigQuery API enabled.
2. ✅ The Vertex AI API enabled.
3. ✅ A BigQuery connection to Vertex AI configured.
4. ✅ Appropriate IAM permissions for your service account.
5. ✅ Access to the MIMIC-IV dataset (or use the sample data provided in this notebook).

In [1]:
%%bigquery

CREATE SCHEMA IF NOT EXISTS `named-storm-470818-t4.patient_semantic_layer`
OPTIONS (
  description = "Healthcare Analytics Semantic Layer - Project Asclepius v2.0",
  location = "US",
  labels = [("environment", "production"), ("project", "asclepius")]
);

Query is running:   0%|          |

""





## get project id using: "gcloud config list"
 - mine is: "named-storm-470818-t4"

-- Create connection to Vertex AI (run via gcloud CLI or Console)
-
```bq mk --connection --display_name='gcp_llm_connection' --connection_type=CLOUD_RESOURCE --project_id=named-storm-470818-t4 --location=US gcp_llm_connection```

-- Grant connection service account Vertex AI User role

gcloud projects add-iam-policy-binding named-storm-470818-t4 \
    --member='serviceAccount:bqcx-533244658486-951a@gcp-sa-bigquery-condel.iam.gserviceaccount.com' \
    --role='roles/aiplatform.user'


     required for it to be able to call the Gemini model endpoint on your behalf.

In [3]:
%%bigquery

-- Create reference to Gemini 1.5 Pro for multimodal reasoning
CREATE OR REPLACE MODEL `named-storm-470818-t4.patient_semantic_layer.llm_model`
REMOTE WITH CONNECTION `named-storm-470818-t4.US.gcp_llm_connection`
OPTIONS (
  ENDPOINT = 'gemini-2.5-pro'
);

Query is running:   0%|          |

""


In [4]:
%%bigquery

-- Create reference to text embedding model for vectorization
CREATE OR REPLACE MODEL `named-storm-470818-t4.patient_semantic_layer.embedding_model`
REMOTE WITH CONNECTION `named-storm-470818-t4.US.gcp_llm_connection`
OPTIONS (
  ENDPOINT = 'text-embedding-004'
);

Query is running:   0%|          |

""


In [5]:
%%bigquery

-- Cell 2.1: Create Sample MIMIC-IV Style Data
-- These queries create the foundational tables with sample patient data for the pipeline.

-- Create patient demographics table
CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.patient_demographics` AS
SELECT * FROM (
  SELECT 100001 as subject_id, 'M' as gender, 65 as anchor_age, 'WHITE' as ethnicity, 'Medicare' as insurance, 'SINGLE' as marital_status
  UNION ALL SELECT 100002, 'F', 72, 'WHITE', 'Medicare', 'MARRIED'
  UNION ALL SELECT 100003, 'M', 80, 'BLACK/AFRICAN AMERICAN', 'Medicare', 'WIDOWED'
  UNION ALL SELECT 100004, 'F', 68, 'HISPANIC/LATINO', 'Medicaid', 'MARRIED'
  UNION ALL SELECT 100005, 'F', 45, 'WHITE', 'Medicaid', 'SINGLE'
  UNION ALL SELECT 100006, 'M', 58, 'ASIAN', 'Private', 'MARRIED'
  UNION ALL SELECT 100007, 'F', 77, 'WHITE', 'Medicare', 'WIDOWED'
  UNION ALL SELECT 100008, 'M', 82, 'WHITE', 'Medicare', 'WIDOWED'
);

Query is running:   0%|          |

""


In [6]:
%%bigquery
-- Create raw clinical notes table
CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.clinical_notes_raw` AS
SELECT * FROM (
  SELECT 'n001' as note_id, 100001 as subject_id, 200001 as hadm_id, 'Discharge summary' as note_type, 1 as note_seq,
    TIMESTAMP('2024-01-15 10:30:00') as charttime,
    'Patient is a 65-year-old male with history of diabetes mellitus type 2, hypertension, and chronic kidney disease admitted for acute exacerbation of heart failure. During hospital stay, patient showed improvement with diuretic therapy. Social history notable for living alone, limited family support. Transportation barriers noted - patient mentions difficulty getting to appointments due to lack of reliable transport. Housing situation stable. Patient discharged home with home health services. Follow-up scheduled with cardiology in 2 weeks. High risk for readmission due to social isolation and medication compliance concerns.' as note_text

  UNION ALL SELECT 'n002', 100001, 200001, 'Progress Notes', 2, TIMESTAMP('2024-01-16 14:20:00'),
    'Progress note day 2: Patient continues to improve, diuresis ongoing. Spoke with case manager regarding discharge planning. Patient expressed concerns about managing medications at home. Lives alone, daughter lives out of state. Limited local support system. Patient mentioned financial constraints affecting medication adherence in the past. Will need close follow-up. Care coordination challenging due to multiple specialists involved - cardiology, nephrology, endocrinology all providing different recommendations.'

  UNION ALL SELECT 'n003', 100002, 200002, 'Discharge summary', 1, TIMESTAMP('2024-02-01 09:15:00'),
    'Discharge Summary: 72-year-old female admitted with pneumonia, complicated by sepsis. Past medical history significant for COPD, diabetes. Patient reports smoking 1 pack per day for 40 years, recently quit. Social determinants assessment: Patient is unemployed, receives disability benefits. Lives in subsidized housing which is stable. Has reliable family support from son who lives nearby. Transportation via medical taxi covered by insurance. Multiple provider handoffs during admission created coordination challenges between emergency medicine, hospitalist, pulmonologist, and infectious disease. Medication reconciliation issues noted. Discharge to home with family support and home health services.'

  UNION ALL SELECT 'n004', 100003, 200003, 'Progress Notes', 1, TIMESTAMP('2024-02-10 16:45:00'),
    'Patient transferred from emergency department after fall at home. 80-year-old male living alone in senior housing. Fall occurred while getting up at night to use bathroom. Home safety evaluation pending. Patient reports irregular meal pattern, relies on delivered meals which are inconsistent. Social isolation noted - minimal contact with family members who live out of state. Mobility issues preventing regular grocery shopping. Physical therapy evaluation recommended. Currently seeing multiple specialists - orthopedics, geriatrics, cardiology - with poor communication between providers. Conflicting medication recommendations noted between providers.'

  UNION ALL SELECT 'n005', 100004, 200004, 'Social Work Notes', 1, TIMESTAMP('2024-03-05 11:30:00'),
    'Social work assessment: Patient is 68-year-old married Hispanic woman with diabetes and hypertension. Significant language barrier noted - limited English proficiency, husband serves as primary translator but not always available. Financial stress reported - medical bills causing severe hardship, considering bankruptcy. Insurance coverage has gaps in specialty care access, leading to delayed treatments. Transportation barriers to appointments due to unreliable vehicle and limited public transit. Lives in stable housing with extended family providing support. Strong cultural/family support system despite financial constraints. Recommended financial counseling and interpreter services.'

  UNION ALL SELECT 'n006', 100005, 200005, 'Nursing Notes', 1, TIMESTAMP('2024-03-12 08:15:00'),
    'Nursing assessment: Patient appears extremely anxious about discharge planning. Reports significant concerns about managing complex care regimen at home. Lives alone with two children ages 8 and 12, serves as single mother. Recently lost employment due to frequent hospitalizations related to chronic conditions. Housing instability - currently behind on rent payments, at risk of eviction within 30 days. Children staying with maternal grandmother during current hospitalization. Patient reports food insecurity, family relies on food bank and school meal programs. Medicaid coverage with significant gaps in specialty care access and prescription coverage. Multiple care coordination challenges identified across primary care, specialty care, and social services.'

  UNION ALL SELECT 'n007', 100006, 200006, 'History and Physical', 1, TIMESTAMP('2024-04-01 20:00:00'),
    'History and Physical: 58-year-old construction worker presenting with chest pain for evaluation of possible myocardial infarction. Ruled out after comprehensive workup. Patient reports work-related stress due to demanding physical labor and job insecurity. Lives with spouse and two teenage children in stable single-family home. Has excellent insurance coverage through union-negotiated plan with comprehensive benefits. Strong social support network including extended family in local area. No significant social determinants barriers identified. Patient has reliable transportation and financial resources. Able to follow up with all recommended appointments. Well-coordinated care team with established relationships between providers.'

  UNION ALL SELECT 'n008', 100007, 200007, 'Progress Notes', 1, TIMESTAMP('2024-04-15 13:30:00'),
    'Progress Note: 77-year-old widow admitted for heart failure exacerbation, third admission in past 6 months. Lives alone in senior housing with minimal family support - one daughter who lives across country and visits rarely. Limited mobility due to severe arthritis restricting activities of daily living. Relies on Meals on Wheels for nutrition support but reports meals are often inadequate. Transportation provided by senior services but availability is limited and unreliable. Social isolation is concerning - reports minimal meaningful social contact, spends most days alone watching television. Multiple hospitalizations suggest significant care fragmentation - seeing cardiologist at Hospital A, primary care at Clinic B, and endocrinologist at Hospital C with poor coordination between sites. Medication reconciliation issues noted with conflicting prescriptions between providers. Recent insurance changes have disrupted established care relationships.'

  UNION ALL SELECT 'n009', 100008, 200008, 'Discharge summary', 1, TIMESTAMP('2024-05-10 11:45:00'),
    'Discharge Summary: 82-year-old widowed male admitted with acute kidney injury and dehydration. Lives alone in two-story home that is increasingly difficult to maintain. Reports skipping medications due to cost constraints despite Medicare coverage - confusion about Part D coverage and prior authorizations. Multiple medication management issues with complex regimen from different specialists. Transportation barriers - no longer drives, relies on neighbors for appointments but feels like a burden. Social isolation severe - wife passed away 2 years ago, limited contact with adult children who live in different states. Home safety concerns - recent falls, clutter, inadequate lighting. Nutrition poor - relies on processed foods, limited cooking ability. Care fragmentation significant - seeing nephrologist, cardiologist, urologist, ophthalmologist at different health systems with minimal communication. Discharge planning complicated by lack of social support and multiple barriers to care access.'
);



Query is running:   0%|          |

""


In [7]:
%%bigquery

-- Create lab results table for structured data
CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.lab_results` AS
SELECT * FROM (
  SELECT 100001 as subject_id, 200001 as hadm_id, 'GLUCOSE' as itemid, 180.5 as valuenum, 'mg/dL' as valueuom, TIMESTAMP('2024-01-15 08:00:00') as charttime
  UNION ALL SELECT 100001, 200001, 'HBA1C', 8.2, '%', TIMESTAMP('2024-01-15 08:00:00')
  UNION ALL SELECT 100001, 200001, 'CREATININE', 1.8, 'mg/dL', TIMESTAMP('2024-01-15 08:00:00')
  UNION ALL SELECT 100002, 200002, 'WBC', 15.2, 'K/uL', TIMESTAMP('2024-02-01 06:00:00')
  UNION ALL SELECT 100002, 200002, 'PROCALCITONIN', 2.4, 'ng/mL', TIMESTAMP('2024-02-01 06:00:00')
  UNION ALL SELECT 100003, 200003, 'HEMOGLOBIN', 9.8, 'g/dL', TIMESTAMP('2024-02-10 07:30:00')
);

Query is running:   0%|          |

""


In [8]:
%%bigquery
-- EHR Note Structuring with ML.GENERATE_TEXT

CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.ehr_structured_extracts` AS
WITH prompts AS (
  SELECT
    note_id,
    subject_id,
    hadm_id,
    note_type,
    note_seq,
    charttime,
    note_text,
    CONCAT(
      'Extract clinical information from this note in JSON format. ',
      'Identify and categorize: ',
      '1) SOCIAL_DETERMINANTS (housing, transportation, employment, social_support, food_security) with severity level (none/mild/moderate/severe) and specific mentions, ',
      '2) CARE_FRAGMENTATION (provider_count, coordination_issues, handoff_problems, medication_conflicts) with severity, ',
      '3) CLINICAL_ENTITIES (diagnoses, medications, procedures, labs) mentioned, ',
      '4) RISK_FACTORS (readmission_risk, complexity_level, medication_adherence_concerns) with assessment. ',
      'Return valid JSON only. Clinical note: ',
      note_text
    ) AS prompt
  FROM `named-storm-470818-t4.patient_semantic_layer.clinical_notes_raw`
)

SELECT
  p.note_id,
  p.subject_id,
  p.hadm_id,
  p.note_type,
  p.note_seq,
  p.charttime,
  p.note_text AS original_text,
  g.ml_generate_text_llm_result as structured_extract
FROM
  ML.GENERATE_TEXT(
    MODEL `named-storm-470818-t4.patient_semantic_layer.llm_model`,
    (
      SELECT note_id, prompt FROM prompts
    ),
    STRUCT(
      0.1 AS temperature,
      2048 AS max_output_tokens,
      TRUE AS flatten_json_output
    )
  ) AS g
JOIN prompts AS p
ON g.note_id = p.note_id;




Query is running:   0%|          |

""


In [9]:
%%bigquery
select * from named-storm-470818-t4.patient_semantic_layer.ehr_structured_extracts

Query is running:   0%|          |

Downloading:   0%|          |

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,original_text,structured_extract
0,n003,100002,200002,Discharge summary,1,2024-02-01 09:15:00+00:00,Discharge Summary: 72-year-old female admitted...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."
1,n001,100001,200001,Discharge summary,1,2024-01-15 10:30:00+00:00,Patient is a 65-year-old male with history of ...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."
2,n009,100008,200008,Discharge summary,1,2024-05-10 11:45:00+00:00,Discharge Summary: 82-year-old widowed male ad...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."
3,n007,100006,200006,History and Physical,1,2024-04-01 20:00:00+00:00,History and Physical: 58-year-old construction...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."
4,n006,100005,200005,Nursing Notes,1,2024-03-12 08:15:00+00:00,Nursing assessment: Patient appears extremely ...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."
5,n002,100001,200001,Progress Notes,2,2024-01-16 14:20:00+00:00,Progress note day 2: Patient continues to impr...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."
6,n004,100003,200003,Progress Notes,1,2024-02-10 16:45:00+00:00,Patient transferred from emergency department ...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."
7,n008,100007,200007,Progress Notes,1,2024-04-15 13:30:00+00:00,Progress Note: 77-year-old widow admitted for ...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."
8,n005,100004,200004,Social Work Notes,1,2024-03-05 11:30:00+00:00,Social work assessment: Patient is 68-year-old...,"```json\n{\n ""SOCIAL_DETERMINANTS"": {\n ""h..."


In [10]:
%%bigquery

-- Cell 2.3: Create Unified Patient Event Stream
-- This combines all data sources (notes, labs, demographics) into a single, time-ordered event log for each patient.


CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.patient_event_stream` AS

-- Clinical notes events
SELECT
  subject_id,
  hadm_id,
  charttime,
  'CLINICAL_NOTE' as event_type,
  note_type as event_subtype,
  note_id as event_id,
  STRUCT(
    original_text as text_content,
    structured_extract as structured_data
  ) as event_data
FROM `named-storm-470818-t4.patient_semantic_layer.ehr_structured_extracts`

UNION ALL

-- Lab results events
SELECT
  subject_id,
  hadm_id,
  charttime,
  'LAB_RESULT' as event_type,
  itemid as event_subtype,
  CAST(subject_id as STRING) || '_' || itemid || '_' || CAST(charttime as STRING) as event_id,
  STRUCT(
  CAST(valuenum AS STRING) AS text_content,
  TO_JSON_STRING(
    JSON_OBJECT('value', valuenum, 'unit', valueuom, 'item', itemid)
  ) AS structured_data
) AS event_data
FROM `named-storm-470818-t4.patient_semantic_layer.lab_results`

UNION ALL

-- Demographics events (admission-based)
SELECT
  d.subject_id,
  cn.hadm_id,
  cn.charttime,
  'DEMOGRAPHICS' as event_type,
  'PATIENT_INFO' as event_subtype,
  CAST(d.subject_id as STRING) || '_demographics' as event_id,
  STRUCT(
  CONCAT(
    'Patient demographics: ', d.gender, ', age ', d.anchor_age,
    ', ethnicity ', d.ethnicity, ', insurance ', d.insurance
  ) AS text_content,
  TO_JSON_STRING(
    JSON_OBJECT(
      'gender', d.gender,
      'age', d.anchor_age,
      'ethnicity', d.ethnicity,
      'insurance', d.insurance,
      'marital_status', d.marital_status
    )
  ) AS structured_data
) AS event_data
FROM `named-storm-470818-t4.patient_semantic_layer.patient_demographics` d
JOIN (
  SELECT DISTINCT subject_id, hadm_id, MIN(charttime) as charttime
  FROM `named-storm-470818-t4.patient_semantic_layer.clinical_notes_raw`
  GROUP BY subject_id, hadm_id
) cn
  ON d.subject_id = cn.subject_id;


Query is running:   0%|          |

""


In [11]:
%%bigquery

select * from named-storm-470818-t4.patient_semantic_layer.patient_event_stream

Query is running:   0%|          |

Downloading:   0%|          |

,subject_id,hadm_id,charttime,event_type,event_subtype,event_id,event_data
0,100001,200001,2024-01-15 08:00:00+00:00,LAB_RESULT,GLUCOSE,100001_GLUCOSE_2024-01-15 08:00:00+00,"{'text_content': '180.5', 'structured_data': '..."
1,100001,200001,2024-01-15 08:00:00+00:00,LAB_RESULT,CREATININE,100001_CREATININE_2024-01-15 08:00:00+00,"{'text_content': '1.8', 'structured_data': '{""..."
2,100001,200001,2024-01-15 08:00:00+00:00,LAB_RESULT,HBA1C,100001_HBA1C_2024-01-15 08:00:00+00,"{'text_content': '8.2', 'structured_data': '{""..."
3,100001,200001,2024-01-15 10:30:00+00:00,DEMOGRAPHICS,PATIENT_INFO,100001_demographics,"{'text_content': 'Patient demographics: M, age..."
4,100001,200001,2024-01-15 10:30:00+00:00,CLINICAL_NOTE,Discharge summary,n001,{'text_content': 'Patient is a 65-year-old mal...
5,100001,200001,2024-01-16 14:20:00+00:00,CLINICAL_NOTE,Progress Notes,n002,{'text_content': 'Progress note day 2: Patient...
6,100002,200002,2024-02-01 06:00:00+00:00,LAB_RESULT,PROCALCITONIN,100002_PROCALCITONIN_2024-02-01 06:00:00+00,"{'text_content': '2.4', 'structured_data': '{""..."
7,100002,200002,2024-02-01 06:00:00+00:00,LAB_RESULT,WBC,100002_WBC_2024-02-01 06:00:00+00,"{'text_content': '15.2', 'structured_data': '{..."
8,100002,200002,2024-02-01 09:15:00+00:00,CLINICAL_NOTE,Discharge summary,n003,{'text_content': 'Discharge Summary: 72-year-o...
9,100002,200002,2024-02-01 09:15:00+00:00,DEMOGRAPHICS,PATIENT_INFO,100002_demographics,"{'text_content': 'Patient demographics: F, age..."


In [30]:
from google.cloud import bigquery
from google.api_core.exceptions import Conflict
import time

PROJECT_ID = "named-storm-470818-t4"
DATASET = "patient_semantic_layer"
SOURCE_TABLE = f"{PROJECT_ID}.{DATASET}.patient_event_stream"
DEST_TABLE = f"{PROJECT_ID}.{DATASET}.patient_summary_chronicle"
MODEL = f"{PROJECT_ID}.{DATASET}.llm_model"

client = bigquery.Client(project=PROJECT_ID)

# 1. Get unique admissions
admissions_query = f"""
SELECT DISTINCT subject_id, hadm_id
FROM `{SOURCE_TABLE}`
ORDER BY subject_id, hadm_id
"""
admissions = client.query(admissions_query).result()
admission_list = [(row.subject_id, row.hadm_id) for row in admissions]

print(f"Found {len(admission_list)} admissions to process.")

# 2. Ensure destination table exists
schema = [
    bigquery.SchemaField("subject_id", "INT64"),
    bigquery.SchemaField("hadm_id", "INT64"),
    bigquery.SchemaField("total_events", "INT64"),
    bigquery.SchemaField("admission_start", "TIMESTAMP"),
    bigquery.SchemaField("last_event_time", "TIMESTAMP"),
    bigquery.SchemaField("patient_summary", "STRING"),
    bigquery.SchemaField("combined_text_content", "STRING"),
]
table = bigquery.Table(DEST_TABLE, schema=schema)
try:
    client.create_table(table)
    print(f"Created table {DEST_TABLE}")
except Conflict:
    print(f"Table {DEST_TABLE} already exists")

# 3. Process admissions in batches
BATCH_SIZE = 20   # tune this depending on how long each run takes

for i in range(0, len(admission_list), BATCH_SIZE):
    batch = admission_list[i:i+BATCH_SIZE]
    batch_conditions = " OR ".join(
        [f"(subject_id={sid} AND hadm_id={hid})" for sid, hid in batch]
    )

    query = f"""
    INSERT INTO `{DEST_TABLE}`
    SELECT
      subject_id,
      hadm_id,
      total_events,
      admission_start,
      last_event_time,
      JSON_VALUE(ml_generate_text_result, '$.candidates[0].content.parts[0].text') AS patient_summary,
      combined_text_content
    FROM ML.GENERATE_TEXT(
      MODEL `{MODEL}`,
      (
        SELECT
          subject_id,
          hadm_id,
          COUNT(*) AS total_events,
          MIN(charttime) AS admission_start,
          MAX(charttime) AS last_event_time,
          STRING_AGG(COALESCE(event_data.text_content, ''), ' ' ORDER BY charttime) AS combined_text_content,
          CONCAT(
            'Create a comprehensive clinical summary for this patient admission. ',
            'Analyze the complete patient story including: ',
            '1) Clinical condition progression and key events, ',
            '2) Social determinants of health and barriers to care, ',
            '3) Care coordination challenges and provider interactions, ',
            '4) Risk factors and concerns for post-discharge outcomes. ',
            'Synthesize into a coherent narrative under 500 words and be very direct cut all the yap and no extra verbose. We want to save space ',
            'Patient events: ',
            STRING_AGG(
              CONCAT(
                CAST(charttime AS STRING), ' - ', event_type, ': ',
                COALESCE(event_data.text_content, 'No text content')
              ), ' | '
              ORDER BY charttime
            )
          ) AS prompt
        FROM `{SOURCE_TABLE}`
        WHERE {batch_conditions}
        GROUP BY subject_id, hadm_id
      ),
      STRUCT(0.2 AS temperature, 4096 AS max_output_tokens)
    )
    """

    print(f"Processing batch {i//BATCH_SIZE+1} of {len(admission_list)//BATCH_SIZE+1}...")
    job = client.query(query)
    job.result(timeout=3600)  # wait up to 1 hour per batch

    print(f"✅ Completed batch {i//BATCH_SIZE+1}")

    # polite pause to avoid hammering Vertex AI API
    time.sleep(2)


Found 8 admissions to process.
Table named-storm-470818-t4.patient_semantic_layer.patient_summary_chronicle already exists
Processing batch 1 of 1...
✅ Completed batch 1


In [32]:
%%bigquery
select patient_summary from named-storm-470818-t4.patient_semantic_layer.patient_summary_chronicle

Query is running:   0%|          |

Downloading:   0%|          |

,patient_summary
0,**Clinical Admission Summary**\n\n**Patient:**...
1,This clinical summary synthesizes the admissio...
2,This is the third hospital admission in six mo...
3,**Comprehensive Clinical Summary**\n\n**Patien...
4,This is a clinical summary for an 82-year-old ...
5,This is a clinical summary for an 80-year-old ...
6,This clinical summary details the admission of...
7,**Clinical Admission Summary**\n\n**Patient:**...
8,**Clinical Summary**\n\n**Patient:** 58-year-o...
9,**Clinical Admission Summary**\n\nThis is a 68...


# The following cells have not been run yet and contain corrected logic:

In [62]:
%%bigquery
-- Cell 3.2.1: Extract Inferred Social Determinant Burden (ISDB) with Advanced Prompt Engineering
CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.isdb_calculations` AS
SELECT
  subject_id,
  hadm_id,
  ml_generate_text_llm_result AS isdb_analysis,
  source_summary
FROM ML.GENERATE_TEXT(
  MODEL `named-storm-470818-t4.patient_semantic_layer.llm_model`,
  (
    SELECT
      subject_id,
      hadm_id,
      patient_summary AS source_summary,
      -- This REPLACE function injects the patient summary into our robust prompt template
      REPLACE(
        """
        <instructions>
        You are a clinical data extraction expert. Your task is to analyze a patient's clinical summary and generate a structured JSON object containing specific risk scores and determinants.

        Follow these steps carefully:
        1. Read the <patient_summary> provided below.
        2. Analyze the text to identify factors related to the five social determinant categories: housing, transportation, employment, social_support, and food_security.
        3. For each category, provide a 'severity_score' from 0 (none) to 3 (severe) and an 'impact_level' string.
        4. Based on your analysis, calculate an 'overall_isdb_score' on a scale of 0.0 to 1.0.
        5. You MUST return your final answer formatted as a single, valid JSON object that strictly adheres to the schema defined in <json_schema>. Do NOT include any other text, explanations, or markdown formatting like ```json.
        </instructions>

        <json_schema>
        {
          "social_determinants_analysis": {
            "housing": { "severity_score": int, "impact_level": "string" },
            "transportation": { "severity_score": int, "impact_level": "string" },
            "employment": { "severity_score": int, "impact_level": "string" },
            "social_support": { "severity_score": int, "impact_level": "string" },
            "food_security": { "severity_score": int, "impact_level": "string" },
            "overall_isdb_score": float,
            "burden_level": "string"
          }
        }
        </json_schema>

        <example>
          <patient_summary>
            Patient lives alone and mentions it's hard to get to the store. Daughter lives out of state. Recently lost job.
          </patient_summary>
          <output>
            {
              "social_determinants_analysis": {
                "housing": { "severity_score": 0, "impact_level": "none" },
                "transportation": { "severity_score": 2, "impact_level": "moderate" },
                "employment": { "severity_score": 3, "impact_level": "severe" },
                "social_support": { "severity_score": 2, "impact_level": "moderate" },
                "food_security": { "severity_score": 1, "impact_level": "mild" },
                "overall_isdb_score": 0.65,
                "burden_level": "high"
              }
            }
          </output>
        </example>

        <patient_summary>
          {{patient_summary_text}}
        </patient_summary>
        """,
        "{{patient_summary_text}}",
        patient_summary
      ) AS prompt
    FROM
      `named-storm-470818-t4.patient_semantic_layer.patient_summary_chronicle`
  ),
  STRUCT(
    0.1 AS temperature,
    4096 AS max_output_tokens,
    TRUE AS flatten_json_output
  )
);

Query is running:   0%|          |

""


In [63]:
%%bigquery
select * from named-storm-470818-t4.patient_semantic_layer.isdb_calculations

Query is running:   0%|          |

Downloading:   0%|          |

,subject_id,hadm_id,isdb_analysis,source_summary
0,100001,200001,"```json\n{\n ""social_determinants_analysis"": ...",**Clinical Admission Summary**\n\n**Patient:**...
1,100001,200001,"```json\n{\n ""social_determinants_analysis"": ...",This clinical summary synthesizes the admissio...
2,100001,200001,"```json\n{\n ""social_determinants_analysis"": ...",**Clinical Summary**\n\nThis is a 65-year-old ...
3,100002,200002,"```json\n{\n ""social_determinants_analysis"": ...",This clinical summary details the admission of...
4,100002,200002,"```json\n{\n ""social_determinants_analysis"": ...",**Clinical Summary & Analysis**\n\n**Patient:*...
5,100002,200002,"```json\n{\n ""social_determinants_analysis"": ...",This clinical summary details the admission of...
6,100003,200003,"```json\n{\n ""social_determinants_analysis"": ...",This is a clinical summary for an 80-year-old ...
7,100003,200003,"```json\n{\n ""social_determinants_analysis"": ...",**Clinical Admission Summary**\n\n**Patient:**...
8,100003,200003,"```json\n{\n ""social_determinants_analysis"": ...",**Clinical Admission Summary**\n\n**Patient:**...
9,100004,200004,"```json\n{\n ""social_determinants_analysis"": ...",**Comprehensive Clinical Summary**\n\n**Patien...


In [67]:
import logging
import time
from concurrent.futures import ThreadPoolExecutor
from google.cloud import bigquery
from google.api_core.exceptions import Conflict, NotFound, DeadlineExceeded

# --- Configuration ---
PROJECT_ID = "named-storm-470818-t4"
DATASET_ID = "patient_semantic_layer"

# --- Source and Destination Tables ---
SOURCE_TABLE = f"{PROJECT_ID}.{DATASET_ID}.patient_summary_chronicle"
DEST_TABLE = f"{PROJECT_ID}.{DATASET_ID}.cfi_calculations"
CONTROL_TABLE = f"{PROJECT_ID}.{DATASET_ID}.etl_control_cfi" # Tracks job state

# --- Model and Processing Parameters ---
MODEL = f"{PROJECT_ID}.{DATASET_ID}.llm_model"
BATCH_SIZE = 2           # Number of summaries per BQ job. Keep this small to avoid timeouts.
MAX_WORKERS = 5          # Number of BQ jobs to run in parallel.

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
client = bigquery.Client(project=PROJECT_ID)

# --- Prompt Template ---
PROMPT_TEMPLATE = """
<instructions>
You are a clinical data extraction expert. Your task is to analyze a patient's clinical summary for signs of care fragmentation and generate a structured JSON object with relevant scores.
Follow these steps carefully:
1. Read the <patient_summary> provided below.
2. Analyze the text for indicators of care fragmentation, such as multiple uncoordinated providers, communication breakdowns, conflicting advice, or issues during care transitions (handoffs).
3. Based on your analysis, calculate the various scores on a scale of 0.0 to 1.0.
4. You MUST return your final answer formatted as a single, valid JSON object that strictly adheres to the schema defined in <json_schema>. Do NOT include any other text, explanations, or markdown formatting like ```json.
</instructions>
<json_schema>
{
  "care_fragmentation_analysis": {
    "provider_count": int,
    "provider_dispersion_score": float,
    "communication_breakdown_score": float,
    "transition_problems_score": float,
    "overall_cfi_score": float,
    "fragmentation_level": "low|moderate|high"
  }
}
</json_schema>
<example>
  <patient_summary>
    Patient sees a cardiologist at Hospital A and a primary care doctor at Clinic B. Records were not shared between the two, leading to conflicting medication recommendations. The discharge from the ER to the floor was rushed and information was lost.
  </patient_summary>
  <output>
    {
      "care_fragmentation_analysis": {
        "provider_count": 3,
        "provider_dispersion_score": 0.7,
        "communication_breakdown_score": 0.8,
        "transition_problems_score": 0.6,
        "overall_cfi_score": 0.75,
        "fragmentation_level": "high"
      }
    }
  </output>
</example>
<patient_summary>
  {patient_summary_text}
</patient_summary>
"""

def initialize_control_table():
    """Creates and populates a control table with summaries that need processing."""
    logging.info(f"Initializing control table: {CONTROL_TABLE}")
    control_schema = [
        bigquery.SchemaField("subject_id", "INT64", mode="REQUIRED"),
        bigquery.SchemaField("hadm_id", "INT64", mode="REQUIRED"),
        bigquery.SchemaField("status", "STRING", mode="REQUIRED", default_value_expression="'pending'"),
    ]
    try:
        table = bigquery.Table(CONTROL_TABLE, schema=control_schema)
        client.create_table(table)
    except Conflict:
        pass # Table already exists

    populate_sql = f"""
    INSERT INTO `{CONTROL_TABLE}` (subject_id, hadm_id, status)
    SELECT
        source.subject_id,
        source.hadm_id,
        'pending'
    FROM `{SOURCE_TABLE}` AS source
    LEFT JOIN `{DEST_TABLE}` AS dest ON source.subject_id = dest.subject_id AND source.hadm_id = dest.hadm_id
    LEFT JOIN `{CONTROL_TABLE}` AS control ON source.subject_id = control.subject_id AND source.hadm_id = control.hadm_id
    WHERE dest.subject_id IS NULL AND control.subject_id IS NULL;
    """
    job = client.query(populate_sql)
    job.result()
    logging.info(f"{job.num_dml_affected_rows} new summaries added to control table.")

def get_and_lock_batch():
    """
    Selects a batch of pending tasks and updates their status to 'running'.
    This is done in two steps to be compatible with BigQuery's SQL dialect.
    """
    # Step 1: Select a batch of pending rows to determine the work to be done.
    select_sql = f"""
        SELECT subject_id, hadm_id
        FROM `{CONTROL_TABLE}`
        WHERE status = 'pending'
        LIMIT {BATCH_SIZE};
    """
    try:
        query_job = client.query(select_sql)
        results = list(query_job.result()) # Fetch all results

        if not results:
            return [] # No work to do

        batch = [(row.subject_id, row.hadm_id) for row in results]

        # Step 2: Update the status of only the rows we just selected.
        batch_conditions = " OR ".join(
            [f"(subject_id = {sid} AND hadm_id = {hid})" for sid, hid in batch]
        )

        update_sql = f"""
            UPDATE `{CONTROL_TABLE}`
            SET status = 'running', last_updated = CURRENT_TIMESTAMP()
            WHERE {batch_conditions};
        """

        update_job = client.query(update_sql)
        update_job.result() # Wait for the update to complete

        logging.info(f"Locked {len(batch)} items for processing.")
        return batch

    except Exception as e:
        logging.error(f"Failed to get and lock batch: {e}")
        return []

def process_batch(batch):
    """Runs the BigQuery ML job for a given batch."""
    if not batch: return

    batch_filter = " OR ".join([f"(subject_id={sid} AND hadm_id={hid})" for sid, hid in batch])

    query = f"""
    INSERT INTO `{DEST_TABLE}` (subject_id, hadm_id, cfi_analysis, source_summary)
    SELECT
      subject_id,
      hadm_id,
      ml_generate_text_llm_result AS cfi_analysis,
      source_summary
    FROM ML.GENERATE_TEXT(
      MODEL `{MODEL}`,
      (
        SELECT
          subject_id,
          hadm_id,
          patient_summary AS source_summary,
          REPLACE(
            '''{PROMPT_TEMPLATE}''',
            '{{patient_summary_text}}',
            patient_summary
          ) AS prompt
        FROM `{SOURCE_TABLE}`
        WHERE {batch_filter}
      ),
      STRUCT(0.1 AS temperature, 3072 AS max_output_tokens, TRUE AS flatten_json_output)
    );
    """
    try:
        client.query(query).result()
        logging.info(f"✅ Successfully processed batch starting with {batch[0]}.")
        update_status(batch, "completed")
    except DeadlineExceeded:
        logging.warning(f"⏰ Timeout on batch {batch[0]}. Marking for retry.")
        update_status(batch, "pending") # Mark to be picked up again
    except Exception as e:
        logging.error(f"❌ Failed batch {batch[0]}: {e}")
        update_status(batch, "failed")

def update_status(batch, status):
    """Updates the status of a batch in the control table."""
    if not batch: return
    batch_filter = " OR ".join([f"(subject_id={sid} AND hadm_id={hid})" for sid, hid in batch])
    update_sql = f"UPDATE `{CONTROL_TABLE}` SET status = '{status}' WHERE {batch_filter};"
    client.query(update_sql).result()

def main():
    """Main function to run the ETL process."""
    try:
        client.get_table(DEST_TABLE)
    except NotFound:
         logging.warning(f"Destination table {DEST_TABLE} not found. The first INSERT job will create it.")
         # You might need to define the schema if the first job creates the table
         # For simplicity, we assume it can be inferred or already exists.

    initialize_control_table()

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        while True:
            batch = get_and_lock_batch()
            if not batch:
                logging.info("No more pending items.")
                break
            executor.submit(process_batch, batch)
    logging.info("All processing tasks submitted.")

if __name__ == "__main__":
    main()

In [68]:
%%bigquery

select * from named-storm-470818-t4.patient_semantic_layer.cfi_calculations

Query is running:   0%|          |

Downloading:   0%|          |

,subject_id,hadm_id,cfi_analysis,source_summary
0,100001,200001,"```json\n{\n ""care_fragmentation_analysis"": {...",**Clinical Admission Summary**\n\n**Patient:**...
1,100001,200001,"```json\n{\n ""care_fragmentation_analysis"": {...",This clinical summary synthesizes the admissio...
2,100001,200001,"```json\n{\n ""care_fragmentation_analysis"": {...",**Clinical Summary**\n\nThis is a 65-year-old ...
3,100002,200002,"```json\n{\n ""care_fragmentation_analysis"": {...",This clinical summary details the admission of...
4,100002,200002,"```json\n{\n ""care_fragmentation_analysis"": {...",This clinical summary details the admission of...
5,100002,200002,"```json\n{\n ""care_fragmentation_analysis"": {...",**Clinical Summary & Analysis**\n\n**Patient:*...
6,100003,200003,"```json\n{\n ""care_fragmentation_analysis"": {...",**Clinical Admission Summary**\n\n**Patient:**...
7,100003,200003,"```json\n{\n ""care_fragmentation_analysis"": {...",**Clinical Admission Summary**\n\n**Patient:**...
8,100003,200003,"```json\n{\n ""care_fragmentation_analysis"": {...",This is a clinical summary for an 80-year-old ...
9,100004,200004,"```json\n{\n ""care_fragmentation_analysis"": {...",**Comprehensive Clinical Summary**\n\n**Patien...


In [44]:
%%bigquery
-- Cell 3.3: Generate Holistic Patient Trajectory Embeddings
CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.patient_trajectory_embeddings` AS
SELECT
  subject_id,
  hadm_id,
  ml_generate_embedding_result AS patient_trajectory_embedding,
  summary_text,
  combined_text_content,
  total_events,
  admission_start,
  last_event_time
FROM ML.GENERATE_EMBEDDING(
  MODEL `named-storm-470818-t4.patient_semantic_layer.embedding_model`,
  (
    SELECT
      subject_id,
      hadm_id,
      CAST(patient_summary AS STRING) AS summary_text, -- Also cast here for the passthrough column
      combined_text_content,
      total_events,
      admission_start,
      last_event_time,
      -- CORRECTED LINE: The 'content' column must be a STRING
      CAST(patient_summary AS STRING) AS content
    FROM
      `named-storm-470818-t4.patient_semantic_layer.patient_summary_chronicle`
  )
);

Query is running:   0%|          |

""


In [45]:
%%bigquery
select * from named-storm-470818-t4.patient_semantic_layer.patient_trajectory_embeddings

Query is running:   0%|          |

Downloading:   0%|          |

,subject_id,hadm_id,patient_trajectory_embedding,summary_text,combined_text_content,total_events,admission_start,last_event_time
0,100001,200001,"[0.0728440210223198, 0.03545708209276199, -0.0...",**Clinical Admission Summary**\n\n**Patient:**...,"180.5 1.8 8.2 Patient demographics: M, age 65,...",6,2024-01-15 08:00:00+00:00,2024-01-16 14:20:00+00:00
1,100001,200001,"[0.05205830559134483, 0.010610121302306652, 0....",**Clinical Summary**\n\nThis is a 65-year-old ...,"180.5 1.8 8.2 Patient demographics: M, age 65,...",6,2024-01-15 08:00:00+00:00,2024-01-16 14:20:00+00:00
2,100001,200001,"[0.06772884726524353, 0.020883021876215935, -0...",This clinical summary synthesizes the admissio...,"180.5 1.8 8.2 Patient demographics: M, age 65,...",6,2024-01-15 08:00:00+00:00,2024-01-16 14:20:00+00:00
3,100002,200002,"[0.09405700862407684, 0.046409543603658676, -0...",**Clinical Summary & Analysis**\n\n**Patient:*...,2.4 15.2 Discharge Summary: 72-year-old female...,4,2024-02-01 06:00:00+00:00,2024-02-01 09:15:00+00:00
4,100002,200002,"[0.08886225521564484, 0.034798383712768555, -0...",This clinical summary details the admission of...,2.4 15.2 Discharge Summary: 72-year-old female...,4,2024-02-01 06:00:00+00:00,2024-02-01 09:15:00+00:00
5,100002,200002,"[0.08362928777933121, 0.024722320958971977, -0...",This clinical summary details the admission of...,2.4 15.2 Discharge Summary: 72-year-old female...,4,2024-02-01 06:00:00+00:00,2024-02-01 09:15:00+00:00
6,100003,200003,"[0.11949963122606277, 0.03491060063242912, -0....",**Clinical Admission Summary**\n\n**Patient:**...,"9.8 Patient demographics: M, age 80, ethnicity...",3,2024-02-10 07:30:00+00:00,2024-02-10 16:45:00+00:00
7,100003,200003,"[0.10689343512058258, 0.02834656462073326, -0....",This is a clinical summary for an 80-year-old ...,"9.8 Patient demographics: M, age 80, ethnicity...",3,2024-02-10 07:30:00+00:00,2024-02-10 16:45:00+00:00
8,100003,200003,"[0.12415853887796402, 0.03188241273164749, -0....",**Clinical Admission Summary**\n\n**Patient:**...,"9.8 Patient demographics: M, age 80, ethnicity...",3,2024-02-10 07:30:00+00:00,2024-02-10 16:45:00+00:00
9,100004,200004,"[0.0936359390616417, 0.0070231095887720585, -0...",**Comprehensive Clinical Summary**\n\n**Patien...,Social work assessment: Patient is 68-year-old...,2,2024-03-05 11:30:00+00:00,2024-03-05 11:30:00+00:00


In [80]:
%%bigquery

-- STEP 1: Let's look at the raw source data.
SELECT
  subject_id,
  hadm_id,
  isdb_analysis
FROM
  `named-storm-470818-t4.patient_semantic_layer.isdb_calculations`
WHERE
  -- Filter for rows that actually have a JSON object to avoid looking at NULLs
  isdb_analysis IS NOT NULL
LIMIT 1;

Query is running:   0%|          |

Downloading:   0%|          |

,subject_id,hadm_id,isdb_analysis
0,100001,200001,"```json\n{\n ""social_determinants_analysis"": ..."


In [82]:
%%bigquery
-- Cell 4.1: Prepare Training Dataset for Semantic AI Model (Final, with Regex Cleaning)

CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.semantic_training_data` AS
WITH
  llm_outputs_cleaned_and_parsed AS (
    -- Step 1: Use a regular expression to extract the JSON string from the raw model output,
    -- then safely parse it. This is the most robust method.
    SELECT
      subject_id,
      hadm_id,
      SAFE.PARSE_JSON(REGEXP_EXTRACT(isdb_analysis, r'(?s)\{.*\}')) AS parsed_isdb,
      SAFE.PARSE_JSON(REGEXP_EXTRACT(cfi_analysis, r'(?s)\{.*\}')) AS parsed_cfi
    FROM
      `named-storm-470818-t4.patient_semantic_layer.isdb_calculations`
    FULL OUTER JOIN
      `named-storm-470818-t4.patient_semantic_layer.cfi_calculations`
      USING (subject_id, hadm_id)
  )
-- Step 2: Join the clean data with your other tables and extract the final values.
SELECT
  p.subject_id,
  p.hadm_id,
  d.anchor_age,
  d.gender,
  d.ethnicity,
  d.insurance,
  d.marital_status,
  CASE WHEN d.gender = 'M' THEN 1 ELSE 0 END as is_male,
  CASE
    WHEN d.anchor_age >= 80 THEN 'elderly'
    WHEN d.anchor_age >= 65 THEN 'senior'
    WHEN d.anchor_age >= 50 THEN 'middle_aged'
    ELSE 'adult'
  END as age_category,

  -- Extract values from the now-guaranteed-to-be-clean JSON objects
  SAFE_CAST(JSON_VALUE(llm.parsed_isdb, '$.social_determinants_analysis.overall_isdb_score') AS FLOAT64) as isdb_score,
  JSON_VALUE(llm.parsed_isdb, '$.social_determinants_analysis.burden_level') as isdb_burden_level,
  SAFE_CAST(JSON_VALUE(llm.parsed_cfi, '$.care_fragmentation_analysis.overall_cfi_score') AS FLOAT64) as cfi_score,
  JSON_VALUE(llm.parsed_cfi, '$.care_fragmentation_analysis.fragmentation_level') as cfi_fragmentation_level,
  p.total_events,
  DATETIME_DIFF(p.last_event_time, p.admission_start, HOUR) as length_of_stay_hours,

  -- Target label logic
  CASE
    WHEN SAFE_CAST(JSON_VALUE(llm.parsed_isdb, '$.social_determinants_analysis.overall_isdb_score') AS FLOAT64) > 0.5 THEN 1
    WHEN SAFE_CAST(JSON_VALUE(llm.parsed_cfi, '$.care_fragmentation_analysis.overall_cfi_score') AS FLOAT64) > 0.6 THEN 1
    WHEN d.anchor_age > 75 THEN 1
    ELSE 0
  END as adverse_outcome_risk

FROM `named-storm-470818-t4.patient_semantic_layer.patient_summary_chronicle` p
INNER JOIN `named-storm-470818-t4.patient_semantic_layer.patient_demographics` d
  ON p.subject_id = d.subject_id
INNER JOIN
  llm_outputs_cleaned_and_parsed AS llm
  ON p.subject_id = llm.subject_id AND p.hadm_id = llm.hadm_id;

Query is running:   0%|          |

""


In [83]:
%%bigquery
select * from named-storm-470818-t4.patient_semantic_layer.semantic_training_data

Query is running:   0%|          |

Downloading:   0%|          |

,subject_id,hadm_id,anchor_age,gender,ethnicity,insurance,marital_status,is_male,age_category,isdb_score,isdb_burden_level,cfi_score,cfi_fragmentation_level,total_events,length_of_stay_hours,adverse_outcome_risk
0,100005,200005,45,F,WHITE,Medicaid,SINGLE,0,adult,0.80,high,0.87,high,2,0,1
1,100005,200005,45,F,WHITE,Medicaid,SINGLE,0,adult,0.80,high,0.87,high,2,0,1
2,100005,200005,45,F,WHITE,Medicaid,SINGLE,0,adult,0.80,high,0.87,high,2,0,1
3,100005,200005,45,F,WHITE,Medicaid,SINGLE,0,adult,0.73,high,0.87,high,2,0,1
4,100005,200005,45,F,WHITE,Medicaid,SINGLE,0,adult,0.80,high,0.87,high,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,100007,200007,77,F,WHITE,Medicare,WIDOWED,0,senior,0.53,moderate,0.87,high,2,0,1
212,100007,200007,77,F,WHITE,Medicare,WIDOWED,0,senior,0.53,moderate,0.90,high,2,0,1
213,100007,200007,77,F,WHITE,Medicare,WIDOWED,0,senior,0.53,moderate,0.87,high,2,0,1
214,100007,200007,77,F,WHITE,Medicare,WIDOWED,0,senior,0.53,moderate,0.87,high,2,0,1


In [84]:
%%bigquery
SELECT
  adverse_outcome_risk,
  COUNT(*) AS label_count
FROM
  `named-storm-470818-t4.patient_semantic_layer.semantic_training_data`
GROUP BY
  adverse_outcome_risk;

Query is running:   0%|          |

Downloading:   0%|          |

,adverse_outcome_risk,label_count
0,1,189
1,0,27


In [98]:
%%bigquery
-- Cell 4.3: Generate Narrative Risk Score (NRS) with a Heuristic Model
CREATE OR REPLACE TABLE `named-storm-470818-t4.patient_semantic_layer.narrative_risk_scores` AS
SELECT
  p.subject_id,
  p.hadm_id,
  d.anchor_age,
  d.gender,
  d.insurance,

  -- Extract ISDB and CFI scores using the robust parsing logic
  SAFE_CAST(JSON_VALUE(JSON_QUERY(i.isdb_analysis, '$.social_determinants_analysis'), '$.overall_isdb_score') AS FLOAT64) AS social_determinants_component,
  SAFE_CAST(JSON_VALUE(JSON_QUERY(c.cfi_analysis, '$.care_fragmentation_analysis'), '$.overall_cfi_score') AS FLOAT64) AS care_fragmentation_component,

  -- Clinical complexity component (derived from multiple factors)
  LEAST((
    COALESCE(p.total_events, 0) / 20.0 +  -- Normalize by a higher event count
    COALESCE(DATETIME_DIFF(p.last_event_time, p.admission_start, HOUR), 0) / 168.0 + -- Normalize length of stay
    (CASE WHEN d.anchor_age >= 80 THEN 0.3 WHEN d.anchor_age >= 70 THEN 0.2 ELSE 0.1 END) -- Age factor
  ) / 3.0, 1.0) AS clinical_complexity_component,

  -- Define the Narrative Risk Score (NRS) with a simple weighted average
  -- Weights: 40% Social Burden, 30% Care Fragmentation, 30% Clinical Complexity
  (
    COALESCE(SAFE_CAST(JSON_VALUE(JSON_QUERY(i.isdb_analysis, '$.social_determinants_analysis'), '$.overall_isdb_score') AS FLOAT64), 0) * 0.40 +
    COALESCE(SAFE_CAST(JSON_VALUE(JSON_QUERY(c.cfi_analysis, '$.care_fragmentation_analysis'), '$.overall_cfi_score') AS FLOAT64), 0) * 0.30 +
    COALESCE(LEAST((
      COALESCE(p.total_events, 0) / 20.0 +
      COALESCE(DATETIME_DIFF(p.last_event_time, p.admission_start, HOUR), 0) / 168.0 +
      (CASE WHEN d.anchor_age >= 80 THEN 0.3 WHEN d.anchor_age >= 70 THEN 0.2 ELSE 0.1 END)
    ) / 3.0, 1.0), 0) * 0.30
  ) * 100 AS narrative_risk_score,

  CURRENT_TIMESTAMP() as calculated_timestamp

FROM `named-storm-470818-t4.patient_semantic_layer.patient_summary_chronicle` p
INNER JOIN `named-storm-470818-t4.patient_semantic_layer.patient_demographics` d
  ON p.subject_id = d.subject_id
INNER JOIN `named-storm-470818-t4.patient_semantic_layer.isdb_calculations` i
  ON p.subject_id = i.subject_id AND p.hadm_id = i.hadm_id
INNER JOIN `named-storm-470818-t4.patient_semantic_layer.cfi_calculations` c
  ON p.subject_id = c.subject_id AND p.hadm_id = c.hadm_id;

Query is running:   0%|          |

""


In [101]:
%%bigquery
-- Cell 5.1: Clinical Ground Truthing View (Corrected)
CREATE OR REPLACE VIEW `named-storm-470818-t4.patient_semantic_layer.clinical_validation_dashboard` AS
SELECT
  n.subject_id,
  n.hadm_id,
  n.narrative_risk_score,
  -- Add overall_risk_level based on the new score
  CASE
    WHEN n.narrative_risk_score >= 70 THEN 'High Risk'
    WHEN n.narrative_risk_score >= 50 THEN 'Moderate Risk'
    ELSE 'Low Risk'
  END AS overall_risk_level,
  ROUND(n.social_determinants_component * 100, 1) as social_determinants_score,
  ROUND(n.care_fragmentation_component * 100, 1) as care_fragmentation_score,
  ROUND(n.clinical_complexity_component * 100, 1) as clinical_complexity_score,
  -- Correctly parse the final JSON outputs for display
  JSON_VALUE(JSON_QUERY(i.isdb_analysis, '$.social_determinants_analysis.housing'), '$.impact_level') as housing_impact,
  JSON_VALUE(JSON_QUERY(i.isdb_analysis, '$.social_determinants_analysis.transportation'), '$.impact_level') as transportation_impact,
  JSON_VALUE(JSON_QUERY(c.cfi_analysis, '$.care_fragmentation_analysis'), '$.fragmentation_level') as fragmentation_level,
  p.summary_text as clinical_summary_excerpt,
  n.calculated_timestamp
FROM `named-storm-470818-t4.patient_semantic_layer.narrative_risk_scores` n
LEFT JOIN `named-storm-470818-t4.patient_semantic_layer.isdb_calculations` i ON n.subject_id = i.subject_id AND n.hadm_id = i.hadm_id
LEFT JOIN `named-storm-470818-t4.patient_semantic_layer.cfi_calculations` c ON n.subject_id = c.subject_id AND n.hadm_id = c.hadm_id
LEFT JOIN `named-storm-470818-t4.patient_semantic_layer.patient_trajectory_embeddings` p ON n.subject_id = p.subject_id AND n.hadm_id = p.hadm_id;


Query is running:   0%|          |

""


In [107]:
%%bigquery
-- Cell 5.2: Semantic Cohort Discovery Function (Corrected)
CREATE OR REPLACE TABLE FUNCTION `named-storm-470818-t4.patient_semantic_layer.find_similar_patients`(
  query_description STRING, top_k INT64
) AS (
WITH
  query_vector AS (
    -- Step 1: Generate the embedding for the search query
    SELECT ml_generate_embedding_result
    FROM ML.GENERATE_EMBEDDING(
      MODEL `named-storm-470818-t4.patient_semantic_layer.embedding_model`,
      (SELECT query_description AS content)
    )
  ),
  ranked_similarities AS (
    -- Step 2: Calculate similarity for all patients and assign a rank
    SELECT
      base.subject_id,
      base.hadm_id,
      (1 - COSINE_DISTANCE(base.patient_trajectory_embedding, query_vector.ml_generate_embedding_result)) AS similarity_score,
      ROW_NUMBER() OVER(ORDER BY (1 - COSINE_DISTANCE(base.patient_trajectory_embedding, query_vector.ml_generate_embedding_result)) DESC) as rn
    FROM
      `named-storm-470818-t4.patient_semantic_layer.patient_trajectory_embeddings` AS base,
      query_vector
  )
-- Step 3: Join the top-ranked results with other tables and filter
SELECT
  r.subject_id,
  r.hadm_id,
  r.similarity_score,
  n.narrative_risk_score,
  SUBSTR(e.summary_text, 1, 300) AS patient_summary_excerpt
FROM
  ranked_similarities AS r
LEFT JOIN
  `named-storm-470818-t4.patient_semantic_layer.narrative_risk_scores` AS n
  ON r.subject_id = n.subject_id AND r.hadm_id = n.hadm_id
LEFT JOIN
  `named-storm-470818-t4.patient_semantic_layer.patient_trajectory_embeddings` AS e
  ON r.subject_id = e.subject_id AND r.hadm_id = e.hadm_id
WHERE
  r.rn <= top_k -- Use WHERE to filter by rank instead of LIMIT
ORDER BY
  r.similarity_score DESC
);

Query is running:   0%|          |

""


In [108]:
%%bigquery
-- Cell 5.2: Semantic Cohort Discovery Function (Corrected)
CREATE OR REPLACE TABLE FUNCTION `named-storm-470818-t4.patient_semantic_layer.find_similar_patients`(
  query_description STRING, top_k INT64
) AS (
WITH
  query_vector AS (
    -- Step 1: Generate the embedding for the search query
    SELECT ml_generate_embedding_result
    FROM ML.GENERATE_EMBEDDING(
      MODEL `named-storm-470818-t4.patient_semantic_layer.embedding_model`,
      (SELECT query_description AS content)
    )
  ),
  ranked_similarities AS (
    -- Step 2: Calculate similarity for all patients and assign a rank
    SELECT
      base.subject_id,
      base.hadm_id,
      (1 - COSINE_DISTANCE(base.patient_trajectory_embedding, query_vector.ml_generate_embedding_result)) AS similarity_score,
      ROW_NUMBER() OVER(ORDER BY (1 - COSINE_DISTANCE(base.patient_trajectory_embedding, query_vector.ml_generate_embedding_result)) DESC) as rn
    FROM
      `named-storm-470818-t4.patient_semantic_layer.patient_trajectory_embeddings` AS base,
      query_vector
  )
-- Step 3: Join the top-ranked results with other tables and filter
SELECT
  r.subject_id,
  r.hadm_id,
  r.similarity_score,
  n.narrative_risk_score,
  SUBSTR(e.summary_text, 1, 300) AS patient_summary_excerpt
FROM
  ranked_similarities AS r
LEFT JOIN
  `named-storm-470818-t4.patient_semantic_layer.narrative_risk_scores` AS n
  ON r.subject_id = n.subject_id AND r.hadm_id = n.hadm_id
LEFT JOIN
  `named-storm-470818-t4.patient_semantic_layer.patient_trajectory_embeddings` AS e
  ON r.subject_id = e.subject_id AND r.hadm_id = e.hadm_id
WHERE
  r.rn <= top_k -- Use WHERE to filter by rank instead of LIMIT
ORDER BY
  r.similarity_score DESC
);

Query is running:   0%|          |

""


## Execution Instructions

### Phase 1: Environment Setup (Required First)
Run the setup cells to create the schema, data, and configure connections to the Gemini and embedding models in Vertex AI.

### Phase 2: Execute Stages Sequentially
1. **Stage 1: Data Foundation**: Run all data creation queries to generate the sample patient demographics, clinical notes, and lab results.
2. **Stage 2: AI-Powered Structuring**: Execute the queries that use `ML.GENERATE_TEXT` to structure raw notes and create the `patient_event_stream` and `patient_summary_chronicle`. This stage acts as the **AI Architect**.
3. **Stage 3: KPI Generation**: Run the queries to calculate the ISDB and CFI scores. This stage also uses the **AI Architect** pattern to extract specific metrics from the generated summaries.
4. **Stage 4: Semantic Layer and Final KPIs**: Generate vector embeddings using `ML.GENERATE_EMBEDDING` and calculate the final Narrative Risk Score (NRS). This stage implements the **Semantic Detective** pattern.
5. **Stage 5: Operationalization**: Create the final views and the `find_similar_patients` function to enable clinical validation and semantic search.

### Phase 3: Validation and Use Cases

Once the pipeline is complete, you can use the generated assets for powerful clinical analytics.

#### 1. Find Patients Similar to a Complex Case
This query demonstrates the power of **Semantic Search**. It finds patients who are contextually similar to a complex description, going far beyond simple keyword matching.

```sql
-- Example: Find patients similar to an elderly individual with social isolation and care fragmentation.
SELECT * FROM `named-storm-470818-t4.patient_semantic_layer.find_similar_patients`(
  'elderly patient living alone with social isolation, multiple chronic conditions, seeing many different doctors with poor coordination, and transportation barriers to care',
  5
);
```

#### 2. Get a High-Level Population Health Overview
This query provides a dashboard-ready view of the patient population, segmented by the AI-generated Narrative Risk Score.

```sql
-- Comprehensive population health dashboard
SELECT
  overall_risk_level,
  COUNT(*) as patient_count,
  ROUND(AVG(narrative_risk_score), 1) as avg_nrs,
  ROUND(AVG(social_determinants_score), 1) as avg_social_score,
  ROUND(AVG(care_fragmentation_score), 1) as avg_fragmentation_score
FROM `named-storm-470818-t4.patient_semantic_layer.clinical_validation_dashboard`
GROUP BY overall_risk_level
ORDER BY
  CASE overall_risk_level
    WHEN 'High Risk' THEN 1
    WHEN 'Moderate Risk' THEN 2
    ELSE 3
  END;
```

---

## Expected Outcomes

After completing this implementation, you will have:

✅ **A Complete Semantic Layer**: An AI-driven analytics foundation replacing outdated rule-based systems.
✅ **Novel KPIs**: The NRS, CFI, and ISDB, all extracted directly from unstructured clinical narratives.
✅ **Semantic Search**: A function for concept-based patient similarity discovery.
✅ **Actionable Clinical Insights**: Views ready for building clinical decision support dashboards and alerts.
✅ **A Scalable Architecture**: Built entirely on BigQuery for enterprise-ready scale.

## Next Steps for Production

1. **Integrate Real Data**: Replace the sample data with the full MIMIC-IV dataset or your own EHR data.
2. **Clinical Validation**: Review the AI-generated insights with clinicians to refine prompts and models.
3. **Workflow Integration**: Connect the output views with EHR systems and care management platforms via APIs.
4. **Performance Optimization**: Tune queries and consider vector indexes for larger datasets.
5. **Monitoring and Governance**: Set up automated monitoring for model performance and data quality.